# Bad Apple SIREN Training (H=32) with Weight Continuity

Train 658 SIREN neural networks (3->32->32->3) for Bad Apple video playback on Zynq Z7020 FPGA.

**Weight continuity regularization**: Consecutive segments are initialized from the same random
weights and trained with an L2 penalty on weight differences. This keeps segments close in
weight space, enabling smooth alpha-blended morphing on the FPGA (rather than hard cuts).

**Before running:** Make sure runtime is set to **A100 GPU** (Runtime -> Change runtime type -> A100)

**Steps:**
1. Upload `source.mp4` when prompted
2. Frames get extracted automatically
3. Training runs (~20-30 min on A100)
4. Weights are exported as `weights.bin` (single concatenated file) and zipped for download

In [ ]:
# Cell 1: Upload source video and extract frames
import os
from google.colab import files

# Upload source.mp4
print("Upload source.mp4 (Bad Apple, 320x172, 7MB)")
uploaded = files.upload()
assert 'source.mp4' in uploaded, f"Expected source.mp4, got {list(uploaded.keys())}"
print(f"Uploaded {len(uploaded['source.mp4'])} bytes")

# Extract frames
os.makedirs('frames', exist_ok=True)
!ffmpeg -i source.mp4 -vf "scale=320:172" frames/frame_%05d.png -y -loglevel warning
n_frames = len([f for f in os.listdir('frames') if f.endswith('.png')])
print(f"Extracted {n_frames} frames")

In [ ]:
# Cell 2: Batched SIREN training with weight continuity regularization
import math, os, struct, time
import numpy as np
import torch
from pathlib import Path
from PIL import Image

# =========================================================
# Constants
# =========================================================
FRAME_DIR = Path('frames')
WEIGHTS_DIR = Path('weights')
TOTAL_FRAMES = len([f for f in os.listdir('frames') if f.endswith('.png')])
FRAME_W, FRAME_H = 320, 172
ASPECT_Y = FRAME_H / FRAME_W  # 0.5375
FRAMES_PER_SEGMENT = 10
N_SEGMENTS = (TOTAL_FRAMES + FRAMES_PER_SEGMENT - 1) // FRAMES_PER_SEGMENT
FRAC_BITS = 28
Q_SCALE = 1 << FRAC_BITS
HIDDEN = 32
OMEGA_0 = 10.0

# Training hyperparameters
EPOCHS = 5000
LR = 1e-4
SAMPLES = 50000
MINI_BATCH = 10000
EVAL_EVERY = 50

# Weight continuity: L2 penalty on differences between consecutive segments.
# Keeps segments close in weight space for smooth FPGA alpha-blending.
# 0.0 = independent segments (hard cuts only)
# 0.1 = moderate continuity (good starting point)
# 1.0 = very smooth transitions, lower per-segment quality
REG_LAMBDA = 0.1

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"Config: {TOTAL_FRAMES} frames, {N_SEGMENTS} segments, H={HIDDEN}")
print(f"Training: {EPOCHS} epochs, {SAMPLES} samples/seg, mini-batch {MINI_BATCH}")
print(f"Weight continuity: lambda={REG_LAMBDA}")

# =========================================================
# Load ALL frame data into a single GPU tensor
# =========================================================
print(f"\nLoading {TOTAL_FRAMES} frames into GPU...")
t0 = time.time()

all_seg_frames = []
for seg in range(N_SEGMENTS):
    start_frame = seg * FRAMES_PER_SEGMENT + 1
    end_frame = min(start_frame + FRAMES_PER_SEGMENT, TOTAL_FRAMES + 1)
    frames = []
    for i in range(start_frame, end_frame):
        path = FRAME_DIR / f"frame_{i:05d}.png"
        if not path.exists():
            break
        arr = np.array(Image.open(path).convert('L'), dtype=np.float32) / 127.5 - 1.0
        frames.append(arr)
    if not frames:
        break
    while len(frames) < FRAMES_PER_SEGMENT:
        frames.append(frames[-1])
    all_seg_frames.append(np.stack(frames))

n_seg = len(all_seg_frames)
frames_gpu = torch.from_numpy(np.stack(all_seg_frames)).to(device)
del all_seg_frames

x_coords = (torch.arange(FRAME_W, device=device, dtype=torch.float32) / (FRAME_W - 1)) * 2.0 - 1.0
y_coords = (torch.arange(FRAME_H, device=device, dtype=torch.float32) / (FRAME_H - 1)) * 2.0 * ASPECT_Y - ASPECT_Y
t_coords = (torch.arange(FRAMES_PER_SEGMENT, device=device, dtype=torch.float32) / max(FRAMES_PER_SEGMENT - 1, 1)) * 2.0 - 1.0

print(f"Loaded {n_seg} segments in {time.time()-t0:.1f}s")
print(f"  frames_gpu: {frames_gpu.shape} = {frames_gpu.element_size() * frames_gpu.nelement() / 1e6:.0f} MB on GPU")


# =========================================================
# GPU-native sampling
# =========================================================
def sample_gpu(frames_gpu, n_seg, samples_per_seg):
    fi = torch.randint(0, FRAMES_PER_SEGMENT, (n_seg, samples_per_seg), device=device)
    yi = torch.randint(0, FRAME_H, (n_seg, samples_per_seg), device=device)
    xi = torch.randint(0, FRAME_W, (n_seg, samples_per_seg), device=device)
    x = x_coords[xi]
    y = y_coords[yi]
    t = t_coords[fi]
    coords = torch.stack([x, y, t], dim=2)
    seg_idx = torch.arange(n_seg, device=device).unsqueeze(1).expand_as(fi)
    targets = frames_gpu[seg_idx, fi, yi, xi]
    targets_3ch = targets.unsqueeze(2).expand(-1, -1, 3)
    return coords, targets_3ch


# =========================================================
# Batched SIREN with shared initialization
# =========================================================
def init_weights_shared(n_seg, hidden, omega_0, device):
    """Initialize all segments from the SAME random weights.

    Shared init means segments start identical and only diverge as needed
    to fit their individual frame data. Combined with continuity regularization,
    this keeps consecutive segments close in weight space.
    """
    W1_base = torch.empty(1, hidden, 3, device=device).uniform_(-1.0 / 3, 1.0 / 3)
    W1 = W1_base.expand(n_seg, -1, -1).clone()
    b1 = torch.zeros(n_seg, 1, hidden, device=device)

    bound2 = math.sqrt(6.0 / hidden) / omega_0
    W2_base = torch.empty(1, hidden, hidden, device=device).uniform_(-bound2, bound2)
    W2 = W2_base.expand(n_seg, -1, -1).clone()
    b2 = torch.zeros(n_seg, 1, hidden, device=device)

    bound3 = math.sqrt(6.0 / hidden) / omega_0
    W3_base = torch.empty(1, 3, hidden, device=device).uniform_(-bound3, bound3)
    W3 = W3_base.expand(n_seg, -1, -1).clone()
    b3 = torch.zeros(n_seg, 1, 3, device=device)

    params = [W1, b1, W2, b2, W3, b3]
    for p in params:
        p.requires_grad_(True)
    return params


def batched_forward(coords, params, omega_0):
    W1, b1, W2, b2, W3, b3 = params
    h = torch.bmm(coords, W1.transpose(1, 2)) + b1
    h = torch.sin(omega_0 * h)
    h = torch.bmm(h, W2.transpose(1, 2)) + b2
    h = torch.sin(omega_0 * h)
    out = torch.bmm(h, W3.transpose(1, 2)) + b3
    return torch.sin(out)


def weight_continuity_loss(params):
    """L2 penalty on weight differences between consecutive segments.

    Returns mean squared difference across all parameters and all
    consecutive segment pairs. This is the key term that enables
    smooth alpha-blended morphing between segments on FPGA.
    """
    loss = torch.tensor(0.0, device=params[0].device)
    for p in params:
        diff = p[1:] - p[:-1]  # (N_SEG-1, ...)
        loss = loss + (diff ** 2).mean()
    return loss


# =========================================================
# Q4.28 export
# =========================================================
def float_to_q428(val):
    clamped = max(-8.0, min(val, 8.0 - 1.0 / Q_SCALE))
    raw = int(round(clamped * Q_SCALE))
    if raw < 0:
        raw = raw & 0xFFFFFFFF
    return raw


def export_segment_q428(params, seg_idx, omega_0):
    """Export one segment as list of Q4.28 u32 values."""
    W1, b1, W2, b2, W3, b3 = params
    layers = [
        (W1[seg_idx].detach().cpu().numpy() * omega_0,
         b1[seg_idx, 0].detach().cpu().numpy() * omega_0),
        (W2[seg_idx].detach().cpu().numpy() * omega_0,
         b2[seg_idx, 0].detach().cpu().numpy() * omega_0),
        (W3[seg_idx].detach().cpu().numpy(),
         b3[seg_idx, 0].detach().cpu().numpy()),
    ]
    all_vals = []
    for weights, biases in layers:
        for j in range(weights.shape[0]):
            for k in range(weights.shape[1]):
                all_vals.append(float_to_q428(weights[j, k]))
        for j in range(biases.shape[0]):
            all_vals.append(float_to_q428(biases[j]))
    return all_vals


def export_segment_pt(params, seg_idx, omega_0):
    W1, b1, W2, b2, W3, b3 = params
    state_dict = {
        'layers.0.linear.weight': W1[seg_idx].detach().cpu(),
        'layers.0.linear.bias': b1[seg_idx, 0].detach().cpu(),
        'layers.1.linear.weight': W2[seg_idx].detach().cpu(),
        'layers.1.linear.bias': b2[seg_idx, 0].detach().cpu(),
        'output_layer.weight': W3[seg_idx].detach().cpu(),
        'output_layer.bias': b3[seg_idx, 0].detach().cpu(),
    }
    pt_path = WEIGHTS_DIR / f"segment_{seg_idx:03d}.pt"
    torch.save(state_dict, pt_path)
    return pt_path


# =========================================================
# Evaluation (GPU-native)
# =========================================================
def evaluate_psnr(params, frames_gpu, omega_0, device, max_segs=None):
    W1, b1, W2, b2, W3, b3 = params
    n_seg = frames_gpu.shape[0] if max_segs is None else min(max_segs, frames_gpu.shape[0])
    H, W = FRAME_H, FRAME_W
    xx = x_coords.unsqueeze(0).expand(H, -1)
    yy = y_coords.unsqueeze(1).expand(-1, W)
    xy_flat = torch.stack([xx.reshape(-1), yy.reshape(-1)], dim=1)
    n_pixels = H * W
    psnrs = []
    chunk = 64
    for c_start in range(0, n_seg, chunk):
        c_end = min(c_start + chunk, n_seg)
        c_size = c_end - c_start
        seg_mse_sum = torch.zeros(c_size, device=device)
        for fi in range(FRAMES_PER_SEGMENT):
            t_val = t_coords[fi]
            tt = torch.full((n_pixels, 1), t_val, device=device)
            coords = torch.cat([xy_flat, tt], dim=1)
            coords = coords.unsqueeze(0).expand(c_size, -1, -1)
            with torch.no_grad():
                p = [W1[c_start:c_end], b1[c_start:c_end],
                     W2[c_start:c_end], b2[c_start:c_end],
                     W3[c_start:c_end], b3[c_start:c_end]]
                pred = batched_forward(coords, p, omega_0)
            pred_gray = pred[:, :, 0].reshape(c_size, H, W).clamp(-1.0, 1.0)
            gt = frames_gpu[c_start:c_end, fi]
            seg_mse_sum += ((pred_gray - gt) ** 2).mean(dim=(1, 2))
        seg_mse_avg = seg_mse_sum / FRAMES_PER_SEGMENT
        seg_psnr = 10 * torch.log10(4.0 / seg_mse_avg)
        for s in range(c_size):
            psnrs.append((c_start + s, seg_psnr[s].item()))
    return psnrs


# =========================================================
# Main training loop
# =========================================================
WEIGHTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"\nInitializing {n_seg} SIREN networks (3->{HIDDEN}->{HIDDEN}->3)...")
print(f"Using shared initialization (all segments start identical)")
params = init_weights_shared(n_seg, HIDDEN, OMEGA_0, device)
W1, b1, W2, b2, W3, b3 = params

optimizer = torch.optim.Adam(params, lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=EPOCHS, eta_min=LR * 0.01)

# Track global best (not per-segment, since regularization links segments)
best_total_loss = float('inf')
best_params = [p.detach().clone() for p in params]

n_mini = max(1, SAMPLES // MINI_BATCH)
print(f"Training: {EPOCHS} epochs, {SAMPLES} samples/seg, {n_mini} mini-batches")
print(f"Weight continuity: lambda={REG_LAMBDA}")
print()

t_train = time.time()
for epoch in range(EPOCHS):
    coords_all, targets_all = sample_gpu(frames_gpu, n_seg, SAMPLES)

    epoch_mse = 0.0
    epoch_reg = 0.0
    perm = torch.randperm(SAMPLES, device=device)

    for mb in range(n_mini):
        start = mb * MINI_BATCH
        end = min(start + MINI_BATCH, SAMPLES)
        idx = perm[start:end]
        batch_coords = coords_all[:, idx]
        batch_targets = targets_all[:, idx]
        pred = batched_forward(batch_coords, params, OMEGA_0)
        mse_loss = ((pred - batch_targets) ** 2).mean()
        reg_loss = weight_continuity_loss(params)
        loss = mse_loss + REG_LAMBDA * reg_loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_mse += mse_loss.item()
        epoch_reg += reg_loss.item()

    scheduler.step()
    avg_mse = epoch_mse / n_mini
    avg_reg = epoch_reg / n_mini
    avg_total = avg_mse + REG_LAMBDA * avg_reg

    # Track global best
    if avg_total < best_total_loss:
        best_total_loss = avg_total
        for i, p in enumerate(params):
            best_params[i].copy_(p.detach())

    if (epoch + 1) % EVAL_EVERY == 0 or epoch == 0:
        elapsed = time.time() - t_train
        remaining = elapsed / (epoch + 1) * (EPOCHS - epoch - 1)
        print(f"epoch {epoch+1:5d}/{EPOCHS}: mse={avg_mse:.6f} reg={avg_reg:.6f} "
              f"total={avg_total:.6f} lr={scheduler.get_last_lr()[0]:.2e} "
              f"[{elapsed:.0f}s, ~{remaining:.0f}s left]")
        if (epoch + 1) % 500 == 0:
            psnrs = evaluate_psnr(params, frames_gpu, OMEGA_0, device, max_segs=5)
            avg_psnr = np.mean([p for _, p in psnrs])
            print(f"  PSNR (first 5 segs): {avg_psnr:.1f} dB")

total_time = time.time() - t_train
print(f"\nTraining done in {total_time:.0f}s ({total_time/60:.1f} min)")

# Restore best weights
for i, p in enumerate(params):
    p.data.copy_(best_params[i])

# Report weight continuity stats
with torch.no_grad():
    final_reg = weight_continuity_loss(params).item()
    # Per-param max diff between consecutive segments
    max_diffs = []
    for p in params:
        max_diffs.append((p[1:] - p[:-1]).abs().max().item())
    print(f"Final continuity loss: {final_reg:.6f}")
    print(f"Max weight diff between consecutive segments: {max(max_diffs):.4f}")

# Full PSNR evaluation
print("\nEvaluating all segments...")
t_eval = time.time()
psnrs = evaluate_psnr(params, frames_gpu, OMEGA_0, device)
eval_time = time.time() - t_eval
all_psnr = [p for _, p in psnrs]
print(f"PSNR: avg={np.mean(all_psnr):.1f}dB "
      f"min={np.min(all_psnr):.1f}dB max={np.max(all_psnr):.1f}dB "
      f"[{eval_time:.0f}s]")

# Export all segments as concatenated weights.bin + individual .pt files
print(f"\nExporting {n_seg} segments...")
t_export = time.time()
with open(WEIGHTS_DIR / 'weights.bin', 'wb') as f_concat:
    for seg in range(n_seg):
        export_segment_pt(params, seg, OMEGA_0)
        vals = export_segment_q428(params, seg, OMEGA_0)
        for val in vals:
            f_concat.write(struct.pack('<I', val))
export_size = os.path.getsize(WEIGHTS_DIR / 'weights.bin')
print(f"Exported in {time.time()-t_export:.1f}s")
print(f"  weights.bin: {export_size} bytes ({export_size/1e6:.1f} MB)")
print(f"  .pt files: {WEIGHTS_DIR}/segment_*.pt")

print(f"\n{'='*60}")
print(f"Total: {n_seg} segments, {EPOCHS} epochs, lambda={REG_LAMBDA}")
print(f"Time: {total_time:.0f}s training + {eval_time:.0f}s eval")
print(f"PSNR: {np.mean(all_psnr):.1f} dB average")
print(f"Continuity: {final_reg:.6f} (max diff: {max(max_diffs):.4f})")

In [ ]:
# Cell 3: Download weights.bin (single file for FPGA)
import os
from google.colab import files

weights_path = 'weights/weights.bin'
size_mb = os.path.getsize(weights_path) / 1e6
print(f"weights.bin: {size_mb:.1f} MB")
print(f"Copy to SD card root alongside BOOT.bin")

files.download(weights_path)